# Load API Keys 

In [5]:
# !pip3 install python-dotenv

In [3]:
from dotenv import load_dotenv

load_dotenv('../env.sh')

True

In [2]:
from langchain.llms import OpenAI

/Users/amrita/PycharmProjects/LLMs/venv/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


# LangChain 101

## The LLMs

In [3]:
from langchain.llms import OpenAI

In [16]:
llm = OpenAI(model='text-davinci-003', temperature=0.9)

In [17]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Start your workout routine three times per week with a mid-distance run. Alternate between running a 3 mile route at medium intensity one workout and doing hill sprints for a one mile route the next. 

2. After your run, focus on high-intensity interval training (HIIT) to build up your cardiovascular endurance. Begin with a light warm-up, then do a mixture of squats, lunges, burpees, and jumping jacks for 30 seconds each. Rest for 30 seconds between each exercise.

3. After your HIIT session, cool down with a light jog. Alternatively, you can go for a swim or bike ride to change up your routine and give your muscles a break from running.

4. Twice a week, replace your run or HIIT session with a leisurely bike ride or walk to a nearby park or lake. This will help to build up your endurance and help reduce burnout.

5. Finally, be sure to stretch before and after each workout to improve flexibility and reduce risk of injury.


## The Chain

In [4]:
from langchain import prompts, llms, chains 

In [ ]:
llm = llms.OpenAI(model = "text-davinci-003", temperature = 0.9)
prompt = prompts.PromptTemplate(input_variables = ["product"], template = "What is a good name for a company that makes {product}?")

chain = chains.LLMChain(llm=llm, prompt=prompt)

print(chain.run("eco-friendly water bottles"))

## The Memory

In [5]:
from langchain import memory

In [ ]:
llm = llms.OpenAI(model = "text-davinci-003", temperature = 0)
conversation = chains.ConversationChain(llm=llm, verbose=True, memory=memory.ConversationBufferMemory())

conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

print(conversation)

## Deep Lake Vector store

In [12]:
from langchain.embeddings import openai
from langchain.vectorstores.deeplake import DeepLake
from langchain import text_splitter

In [14]:
llm = llms.OpenAI(model="text-davinci-003", temperature=0)
embeddings = openai.OpenAIEmbeddings(model="text-embedding-ada-002")

In [15]:
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]

text_split = text_splitter.RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_split.create_documents(texts)

In [16]:
my_activeloop_org_id = "amrita95"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"

dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [17]:
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Deep Lake Dataset in hub://amrita95/langchain_course_from_zero_to_hero already exists, loading from the storage


In [18]:
db.add_documents(docs)

Dataset(path='hub://amrita95/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['42b14564-a2fc-11ee-a4f3-a683e744668d',
 '42b14636-a2fc-11ee-a4f3-a683e744668d']

In [19]:
retrieval_qa = chains.RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [20]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [21]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [23]:
# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]

docs = text_split.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

/

Dataset(path='hub://amrita95/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['dcf07956-a2fc-11ee-a4f3-a683e744668d',
 'dcf07a6e-a2fc-11ee-a4f3-a683e744668d']

In [24]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


### Google search Tool

In [25]:
from langchain.llms import OpenAI
from langchain.agents import AgentType, load_tools, initialize_agent, Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [26]:
llm = OpenAI(model="text-davinci-003", temperature=0)

In [28]:
## load Google API keys
load_dotenv('../env.sh')

True

In [30]:
!poetry add google-api-python-client

Using version ^2.111.0 for google-api-python-client

Updating dependencies
Resolving dependencies... (6.4s)s://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl (2.9s)

Package operations: 11 installs, 0 updates, 0 removals

  • Installing pyasn1 (0.5.1): Pending...
  • Installing pyasn1 (0.5.1): Downloading... 0%
  • Installing pyasn1 (0.5.1): Downloading... 100%
  • Installing pyasn1 (0.5.1): Installing...
  • Installing pyasn1 (0.5.1)
  • Installing cachetools (5.3.2): Pending...
  • Installing pyasn1-modules (0.3.0): Pending...
  • Installing rsa (4.9): Pending...
  • Installing rsa (4.9): Pending...
  • Installing pyasn1-modules (0.3.0): Downloading... 0%
  • Installing rsa (4.9): Pending...
  • Installing rsa (4.9): Pending...
  • Installing pyasn1-modules (0.3.0): Downloading... 100%
  • Installing rsa (4.9): Pending...
  • Installing rsa (4.9): Pending...
  • Installing pyasn1-modules (0.3.0): In

In [31]:
search = GoogleSearchAPIWrapper()

In [32]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [33]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [37]:
response = agent("What's the latest news about the Mars Rover?")
print(response['output'])



> Entering new  chain...
 I should find out what the latest news is.
Action: google-search
Action Input: "Amrita Veerabaghu news"
Observation: Exciting news - we've joined Instagram!… Exciting news - we've joined Instagram!… ... Amrita Sundari Veerabagu. Software Engineer, Machine Learning at Niantic, ... 2013 – Breaking News Live; 2013 - 3 G; 2013 - Last Bus 8:30 PM; 2014 - Law Point ... Amrita TV, Devan, Malayalam. 2010, Indhraneelam, Surya TV, Malayalam. 2010- ... Amrita Singh. Robert Half University of California, Santa Cruz. San Jose ... Concord, CA · Connect · Amrita Sundari Veerabagu. San Jose, CA · Connect · L. Mattock ... Nov 2, 2023 ... News Edition, American Chemical Society, Organic Letters, Organic ... Amrita V.; Kleinheinz, Tracy; Lai, Tommy; Leipold, Douglas D.; Li, Chun ... Premkumar S. Principal Software Engineer at Optum. Optum Amrita School of Engineering,Coimbatore. Chennai, Tamil Nadu, India. 560 followers 500+ connections. Amrita Vidyalayam. 4.248 Ratings. Marav

## Tools in LangChain

### Google Search + Text Summarizer tool

In [39]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [40]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [41]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [42]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [44]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I need to find the latest news about the Mars rover and then summarize it.
Action: Search
Action Input: Latest news about the Mars rover
Observation: NASA's Curiosity Mars rover captured these clouds just after sunset on March 19, 2021, ... Feb 8, 2023 ... NASA's Curiosity Mars rover has discovered lots of evidence of ancient lakes on the Red Planet – and this recent panorama shows intriguing ... News ; NASA's Perseverance Rover Deciphers Ancient History of Martian Lake. December 12, 2023 ; NASA's MAVEN Observes the Disappearing Solar Wind. December 11, ... Dec 15, 2021 ... These and other findings were presented today during a news briefing at the American Geophysical Union fall science meeting in New Orleans. Even ... NASA's Mars 2020 Perseverance rover will look for signs of past microbial life, cache rock and soil samples, and prepare for future human exploration. Apr 13, 2023 ... NASA's Curiosity Mars rover will drive faster and reduce wear on its wheel

# Few Shot Learning 

In [4]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# load the model
chat = ChatOpenAI(model_name="gpt-4", temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the meaning of life?")

"The meaning of life is a bit like the meaning of a good joke. If you have to explain it, it's not that funny. But I think it's about finding your own punchline."

# Text Summarization, Text Translation, and Question Answering

In [6]:
load_dotenv('../env.sh')

True

In [7]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [8]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

/Users/amrita/PycharmProjects/LLMs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/amrita/PycharmProjects/LLMs/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


paris


## Multiple questions

In [9]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print( res )

generations=[[Generation(text='paris', generation_info=None)], [Generation(text='giraffe', generation_info=None)], [Generation(text='nitrogen', generation_info=None)], [Generation(text='yellow', generation_info=None)]] llm_output=None run=RunInfo(run_id=UUID('c82958ae-b1ee-4240-8475-98e5ddc463c2'))


In [14]:
from langchain.llms import OpenAI

In [15]:
llm = OpenAI(model_name="text-davinci-003")
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

'Paris\nBlue Whale\nNitrogen\nYellow'

## Text Summarization

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)

In [18]:
summarized_text

'LangChain offers various modules for building language model applications, allowing users to combine them for more complex applications or use them individually for simpler ones, with the basic building block being calling an LLM on input, as demonstrated in the example of creating a company name based on its product.'

## Text translation

In [19]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [22]:
source_language = "English"
target_language = "Tamil"
text = "Hello, how are you doing? What did you have for breakfast?"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)

In [23]:
translated_text

'வணக்கம், நீங்கள் எப்படி இருக்கின்றீர்கள்? நீங்கள் காலை உணவுக்காக என்ன சாப்பிட்டீர்கள்? (Vaṇakkam, nīṅkaḷ eppaṭi irukkiṉṟīrkaḷ? Nīṅkaḷ kālai uṇavukkāka eṉṉa sāppiṭṭīrkaḷ?)'

# Tokenization

In [24]:
from transformers import AutoTokenizer

# Download and load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 665/665 [00:00<00:00, 1.45MB/s]
vocab.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.04M/1.04M [00:00<00:00, 4.36MB/s]
merges.txt: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [25]:
print( tokenizer.vocab )

{'CLOSE': 32737, 'ixt': 6346, 'ĠSearch': 11140, 'ĠFerry': 42609, 'cuts': 23779, 'ĠCrom': 39131, 'ern': 1142, "'.": 4458, 'Ġgrap': 23360, 'Ġstatutes': 24895, 'Ġtide': 20013, 'Ġretire': 8058, 'Ġcavity': 31643, 'ĠWorldwide': 33140, 'used': 1484, 'ours': 4662, 'Probably': 34784, ']-': 45297, 'ooks': 31085, 'norm': 27237, 'Ġconcurrently': 47480, 'Ġdecl': 2377, 'cyclopedia': 25497, 'ĠHomeless': 47706, 'Ġsnatched': 43276, 'Ġaspiration': 48217, 'ĠDemocrats': 4956, 'rollers': 36667, 'ĠGodd': 8935, 'Ġrenowned': 22140, 'pants': 38895, 'ĠNew': 968, 'ĠCommonwealth': 17165, 'Ġwarned': 7728, 'ĠLamar': 26907, 'ĠSolo': 20284, 'Ġmerged': 23791, 'Senator': 29774, 'each': 27379, 'ĠThames': 42521, 'Ġcoales': 46064, 'Ultimate': 47892, 'Ġim': 545, 'democracy': 42017, 'Honest': 37411, 'Buffer': 28632, 'Ġtenant': 18285, 'Ġaccustomed': 23840, 'ĠLauder': 39212, 'Ġupbringing': 40678, 'Ġenjoyment': 23334, 'ĠVanity': 46584, 'Ġbiologist': 35261, 'ĠFolder': 48107, 'Ġacademics': 22321, 'Ġfluorescent': 42309, 'holders'

In [26]:
token_ids = tokenizer.encode("This is a sample text to test the tokenizer.")

In [27]:
print( "Tokens:   ", tokenizer.convert_ids_to_tokens( token_ids ) )
print( "Token IDs:", token_ids )

Tokens:    ['This', 'Ġis', 'Ġa', 'Ġsample', 'Ġtext', 'Ġto', 'Ġtest', 'Ġthe', 'Ġtoken', 'izer', '.']
Token IDs: [1212, 318, 257, 6291, 2420, 284, 1332, 262, 11241, 7509, 13]


# LangChain: Prompt, Summarization and Q&A

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())

print(response.content)

"Inception" is a science fiction action film directed by Christopher Nolan. It was released in 2010 and stars Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and Marion Cotillard. The film follows a professional thief who steals information by infiltrating the subconscious of his targets through their dreams. 

The plot revolves around Dom Cobb (played by DiCaprio), who is offered a chance to have his criminal history erased in exchange for performing the act of "inception" - planting an idea in someone's mind rather than stealing it. As Cobb assembles a team to carry out this dangerous mission, they must navigate through various dream levels, each with its own unique challenges and risks.

"Inception" received critical acclaim for its originality, visual effects, and complex storytelling. It was praised for its thought-provoking themes and exploration of the nature of reality and dreams. The film was a commercial success, grossing over $828 million worldwide.

It won f

In [ ]:
# Import necessary modules
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

# Initialize language model
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Load the summarization chain
summarize_chain = load_summarize_chain(llm)

# Load the document using PyPDFLoader
document_loader = PyPDFLoader(file_path="path/to/your/pdf/file.pdf")
document = document_loader.load()

# Summarize the document
summary = summarize_chain(document)
print(summary['output_text'])

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

llm = OpenAI(model_name="text-davinci-003", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

In [31]:
chain.run("what is the meaning of life?")

' The meaning of life is subjective and can vary from person to person. For some, it may be to find happiness and fulfillment, while for others it may be to make a difference in the world. Ultimately, the meaning of life is up to each individual to decide.'

# Chat models

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
  HumanMessage,
  SystemMessage
)

chat = ChatOpenAI(model_name="gpt-4", temperature=0)

messages = [
	SystemMessage(content="You are a helpful assistant that translates English to Tamil."),
	HumanMessage(content="Translate the following sentence: I love you.")
]

chat(messages)

AIMessage(content='நான் உன்னை காதலிக்கிறேன்.', additional_kwargs={}, example=False)

# GPT4 and ChatGPT

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="The capital of France is Paris.")
]

In [36]:
prompt = HumanMessage(
    content="I'd like to know more about the city you just mentioned."
)
# add to messages
messages.append(prompt)

llm = ChatOpenAI(model_name="gpt-4")

response = llm(messages)

In [37]:
response

AIMessage(content='Paris, the capital city of France, is one of the most famous cities in the world. Known as the "City of Light" or the "City of Love," Paris is renowned for its romantic charm. It is situated in the north-central part of the country and is the largest city in France.\n\nParis is celebrated for its beautiful city plan, architecture, museums, bridges, cathedrals, parks, shopping, gastronomic delights, and haute couture. It has an iconic skyline, dominated by the Eiffel Tower, which is one of the most visited monuments in the world.\n\nThe city is also home to world-class museums like The Louvre, which houses thousands of works of art, including the Mona Lisa and the Venus de Milo. The Musee d\'Orsay, another renowned museum in Paris, holds French art dating from 1848 to 1915, including paintings, sculptures, furniture, and photography.\n\nParis is also known for its culinary scene and is famous for its cafes, bakeries, and restaurants. The city is also the backdrop to m